In [80]:
import xgboost as xgb
import pandas as pd
import numpy as np
from sklearn import preprocessing

In [81]:
train = pd.read_csv(r"./dataset/train.csv")
train.head()
"""
    user_id:没有实际意义，只是一个代号
    shop_id:商店的位置，是预测目标
    longitude:商店的经度
    latitude:商店的纬度
    wifi_id:路由器的编号
    wifi_strong:wifi的信号强度
    con_sta:wifi的连接状态
"""

'\n    user_id:没有实际意义，只是一个代号\n    shop_id:商店的位置，是预测目标\n    longitude:商店的经度\n    latitude:商店的纬度\n    wifi_id:路由器的编号\n    wifi_strong:wifi的信号强度\n    con_sta:wifi的连接状态\n'

In [82]:
test = pd.read_csv(r"./dataset/test.csv")
test.head()

,row_id,shop_id,user_id,time_stamp,longitude,latitude,wifi_id1,wifi_strong1,con_sta1,wifi_id2,...,con_sta7,wifi_id8,wifi_strong8,con_sta8,wifi_id9,wifi_strong9,con_sta9,wifi_id10,wifi_strong10,con_sta10
0,118742,NaN,u_30097142,2017/9/5 13:00,122.141011,39.818847,b_34366982,-82,False,b_37756289,...,FALSE,b_28978909,-62.0,FALSE,b_21518966,-68.0,FALSE,b_13748229,-72.0,false\n
1,118743,NaN,u_30097803,2017/9/6 13:10,118.191907,32.855858,b_36722251,-81,False,b_10537579,...,FALSE,b_21694478,-80.0,FALSE,b_44551973,-72.0,FALSE,b_21694477,-85.0,false\n
2,118744,NaN,u_30097889,2017/9/6 17:40,119.192110,32.424667,b_30026291,-74,False,b_30026290,...,FALSE,b_50235613,-75.0,FALSE,b_17955238,-85.0,FALSE,b_40924464,-54.0,false\n
3,118745,NaN,u_30098996,2017/9/3 12:10,120.612201,34.055249,b_33412374,-77,False,b_22084893,...,FALSE,b_21282193,-87.0,FALSE,b_33334040,-71.0,FALSE,b_29623262,-68.0,false\n
4,118746,NaN,u_30099170,2017/9/2 20:40,116.861989,40.326858,b_19882704,-77,False,b_2241462,...,FALSE,b_585687,-57.0,FALSE,b_37967785,-62.0,FALSE,b_29284311,-42.0,false\n


In [83]:
train['time_stamp'] = pd.to_datetime(pd.Series(train['time_stamp']))
test['time_stamp'] = pd.to_datetime(pd.Series(train['time_stamp']))
train.head()

,user_id,shop_id,time_stamp,longitude,latitude,wifi_id1,wifi_strong1,con_sta1,wifi_id2,wifi_strong2,...,con_sta7,wifi_id8,wifi_strong8,con_sta8,wifi_id9,wifi_strong9,con_sta9,wifi_id10,wifi_strong10,con_sta10
0,u_376,s_2871718,2017-08-06 21:20:00,122.308291,32.088040,b_6396480,-67,False,b_41124514,-86,...,FALSE,b_56326644,-89.0,FALSE,b_56328155,-77.0,FALSE,b_5857369,-55.0,false\n
1,u_376,s_2871718,2017-08-06 21:20:00,122.308162,32.087970,b_6396480,-67,False,b_56328155,-73,...,FALSE,b_6396479,-57.0,FALSE,b_31100514,-89.0,FALSE,b_5857369,-57.0,false\n
2,u_1041,s_181637,2017-08-02 13:10:00,117.365255,40.638214,b_8006367,-78,False,b_2485110,-52,...,FALSE,b_8006521,-74.0,FALSE,b_35013153,-56.0,FALSE,b_37608251,-84.0,false\n
3,u_1158,s_609470,2017-08-13 12:30:00,121.134451,31.197416,b_26250579,-73,False,b_26250580,-64,...,FALSE,b_30424471,-60.0,FALSE,b_26250578,-72.0,FALSE,b_29510856,-80.0,false\n
4,u_1654,s_3816766,2017-08-25 19:50:00,122.255867,31.351320,b_39004150,-66,False,b_39004148,-58,...,FALSE,b_6805211,-80.0,FALSE,b_1845687,-72.0,FALSE,b_21685901,-91.0,false\n


In [84]:
train['Year'] = train['time_stamp'].apply(lambda x: x.year)
train['Month'] = train['time_stamp'].apply(lambda x: x.month)
train['weekday'] = train['time_stamp'].dt.dayofweek
train['time'] = train['time_stamp'].dt.time

test['Year'] = test['time_stamp'].apply(lambda x: x.year)
test['Month'] = test['time_stamp'].apply(lambda x: x.month)
test['weekday'] = test['time_stamp'].dt.dayofweek
test['time'] = test['time_stamp'].dt.time

In [85]:
train.head()

,user_id,shop_id,time_stamp,longitude,latitude,wifi_id1,wifi_strong1,con_sta1,wifi_id2,wifi_strong2,...,wifi_id9,wifi_strong9,con_sta9,wifi_id10,wifi_strong10,con_sta10,Year,Month,weekday,time
0,u_376,s_2871718,2017-08-06 21:20:00,122.308291,32.088040,b_6396480,-67,False,b_41124514,-86,...,b_56328155,-77.0,FALSE,b_5857369,-55.0,false\n,2017,8,6,21:20:00
1,u_376,s_2871718,2017-08-06 21:20:00,122.308162,32.087970,b_6396480,-67,False,b_56328155,-73,...,b_31100514,-89.0,FALSE,b_5857369,-57.0,false\n,2017,8,6,21:20:00
2,u_1041,s_181637,2017-08-02 13:10:00,117.365255,40.638214,b_8006367,-78,False,b_2485110,-52,...,b_35013153,-56.0,FALSE,b_37608251,-84.0,false\n,2017,8,2,13:10:00
3,u_1158,s_609470,2017-08-13 12:30:00,121.134451,31.197416,b_26250579,-73,False,b_26250580,-64,...,b_26250578,-72.0,FALSE,b_29510856,-80.0,false\n,2017,8,6,12:30:00
4,u_1654,s_3816766,2017-08-25 19:50:00,122.255867,31.351320,b_39004150,-66,False,b_39004148,-58,...,b_1845687,-72.0,FALSE,b_21685901,-91.0,false\n,2017,8,4,19:50:00


In [86]:
train = train.drop("time_stamp", axis=1)
test = test.drop("time_stamp", axis=1)   # axis=1列方向，水平方向
train = train.dropna(axis=0)
test = test.fillna(method='pad')  # axis=0行方向，垂直方向

In [87]:
train.head()

,user_id,shop_id,longitude,latitude,wifi_id1,wifi_strong1,con_sta1,wifi_id2,wifi_strong2,con_sta2,...,wifi_id9,wifi_strong9,con_sta9,wifi_id10,wifi_strong10,con_sta10,Year,Month,weekday,time
0,u_376,s_2871718,122.308291,32.088040,b_6396480,-67,False,b_41124514,-86,FALSE,...,b_56328155,-77.0,FALSE,b_5857369,-55.0,false\n,2017,8,6,21:20:00
1,u_376,s_2871718,122.308162,32.087970,b_6396480,-67,False,b_56328155,-73,FALSE,...,b_31100514,-89.0,FALSE,b_5857369,-57.0,false\n,2017,8,6,21:20:00
2,u_1041,s_181637,117.365255,40.638214,b_8006367,-78,False,b_2485110,-52,FALSE,...,b_35013153,-56.0,FALSE,b_37608251,-84.0,false\n,2017,8,2,13:10:00
3,u_1158,s_609470,121.134451,31.197416,b_26250579,-73,False,b_26250580,-64,FALSE,...,b_26250578,-72.0,FALSE,b_29510856,-80.0,false\n,2017,8,6,12:30:00
4,u_1654,s_3816766,122.255867,31.351320,b_39004150,-66,False,b_39004148,-58,FALSE,...,b_1845687,-72.0,FALSE,b_21685901,-91.0,false\n,2017,8,4,19:50:00


In [88]:
test.head()

,row_id,shop_id,user_id,longitude,latitude,wifi_id1,wifi_strong1,con_sta1,wifi_id2,wifi_strong2,...,wifi_id9,wifi_strong9,con_sta9,wifi_id10,wifi_strong10,con_sta10,Year,Month,weekday,time
0,118742,NaN,u_30097142,122.141011,39.818847,b_34366982,-82,False,b_37756289,-53,...,b_21518966,-68.0,FALSE,b_13748229,-72.0,false\n,2017,8,6,21:20:00
1,118743,NaN,u_30097803,118.191907,32.855858,b_36722251,-81,False,b_10537579,-75,...,b_44551973,-72.0,FALSE,b_21694477,-85.0,false\n,2017,8,6,21:20:00
2,118744,NaN,u_30097889,119.192110,32.424667,b_30026291,-74,False,b_30026290,-74,...,b_17955238,-85.0,FALSE,b_40924464,-54.0,false\n,2017,8,2,13:10:00
3,118745,NaN,u_30098996,120.612201,34.055249,b_33412374,-77,False,b_22084893,-86,...,b_33334040,-71.0,FALSE,b_29623262,-68.0,false\n,2017,8,6,12:30:00
4,118746,NaN,u_30099170,116.861989,40.326858,b_19882704,-77,False,b_2241462,-49,...,b_37967785,-62.0,FALSE,b_29284311,-42.0,false\n,2017,8,4,19:50:00


In [42]:
# from sklearn.base import BaseEstimator, TransformerMixin

# class DataFrameImputer(TransformerMixin):
#     def fit(self, X,y=None):
#         for c in X:
#             if X[c].dtype == np.dtype('O'):
#                 fill_number = X[c].value_counts().index[0]
#                 self.fill = pd.Series(fill_number, index=X.columns)
#             else:
#                 fill_number = X[c].median()
#                 self.fill = pd.Series(fill_number, index=X.columns)
#         return self
    
#     def transform(self, X, y=None):
#         return X.fillna(self.fill)

In [49]:
# train = DataFrameImputer().fit_transform(train)
# train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 467 entries, 0 to 499
Data columns (total 38 columns):
user_id          467 non-null object
shop_id          467 non-null object
longitude        467 non-null float64
latitude         467 non-null float64
wifi_id1         467 non-null object
wifi_strong1     467 non-null int64
con_sta1         467 non-null bool
wifi_id2         467 non-null object
wifi_strong2     467 non-null int64
con_sta2         467 non-null object
wifi_id3         467 non-null object
wifi_strong3     467 non-null float64
con_sta3         467 non-null object
wifi_id4         467 non-null object
wifi_strong4     467 non-null float64
con_sta4         467 non-null object
wifi_id5         467 non-null object
wifi_strong5     467 non-null float64
con_sta5         467 non-null object
wifi_id6         467 non-null object
wifi_strong6     467 non-null float64
con_sta6         467 non-null object
wifi_id7         467 non-null object
wifi_strong7     467 non-null float64
con_

In [55]:
# test = DataFrameImputer().fit_transform(test)
# test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 467 entries, 0 to 499
Data columns (total 38 columns):
user_id          467 non-null int64
shop_id          467 non-null int64
longitude        467 non-null float64
latitude         467 non-null float64
wifi_id1         467 non-null int64
wifi_strong1     467 non-null int64
con_sta1         467 non-null bool
wifi_id2         467 non-null int64
wifi_strong2     467 non-null int64
con_sta2         467 non-null int64
wifi_id3         467 non-null int64
wifi_strong3     467 non-null float64
con_sta3         467 non-null int64
wifi_id4         467 non-null int64
wifi_strong4     467 non-null float64
con_sta4         467 non-null int64
wifi_id5         467 non-null int64
wifi_strong5     467 non-null float64
con_sta5         467 non-null int64
wifi_id6         467 non-null int64
wifi_strong6     467 non-null float64
con_sta6         467 non-null int64
wifi_id7         467 non-null int64
wifi_strong7     467 non-null float64
con_sta7         4

In [89]:
for f in train.columns:
    if train[f].dtype == 'object':
        if f != 'shop_id':
            print("train col:\n", f)
            labels = preprocessing.LabelEncoder()
            train[f] = labels.fit_transform(list(train[f].values))
for f in test.columns:
    if test[f].dtype == 'object':
        print("test col:\n", f)
        labels = preprocessing.LabelEncoder()
        labels.fit(list(test[f].values))
        test[f] = labels.transform(list(test[f].values))

col:
 user_id
col:
 wifi_id1
col:
 wifi_id2
col:
 con_sta2
col:
 wifi_id3
col:
 con_sta3
col:
 wifi_id4
col:
 con_sta4
col:
 wifi_id5
col:
 con_sta5
col:
 wifi_id6
col:
 con_sta6
col:
 wifi_id7
col:
 con_sta7
col:
 wifi_id8
col:
 con_sta8
col:
 wifi_id9
col:
 con_sta9
col:
 wifi_id10
col:
 con_sta10
col:
 time
col:
 user_id
col:
 wifi_id1
col:
 wifi_id2
col:
 wifi_id3
col:
 con_sta3
col:
 wifi_id4
col:
 wifi_id5
col:
 con_sta5
col:
 wifi_id6
col:
 wifi_id7
col:
 con_sta7
col:
 wifi_id8
col:
 con_sta8
col:
 wifi_id9
col:
 con_sta9
col:
 wifi_id10
col:
 con_sta10
col:
 time


In [90]:
feature_columns_to_use = ['Year', 'Month', 'weekday',
'time', 'longitude', 'latitude',
'wifi_id1', 'wifi_strong1', 'con_sta1',
 'wifi_id2', 'wifi_strong2', 'con_sta2',
'wifi_id3', 'wifi_strong3', 'con_sta3',
'wifi_id4', 'wifi_strong4', 'con_sta4',
'wifi_id5', 'wifi_strong5', 'con_sta5',
'wifi_id6', 'wifi_strong6', 'con_sta6',
'wifi_id7', 'wifi_strong7', 'con_sta7',
'wifi_id8', 'wifi_strong8', 'con_sta8',
'wifi_id9', 'wifi_strong9', 'con_sta9',
'wifi_id10', 'wifi_strong10', 'con_sta10',]

In [91]:
big_train = train[feature_columns_to_use]
big_test = test[feature_columns_to_use]

train_X = big_train.as_matrix()
test_X = big_test.as_matrix()

train_y = train['shop_id']

In [92]:
# xgboost生成的决策树，处理的特征都是连续型的！！
gbm = xgb.XGBClassifier(silent=1,max_depth=10, n_estimators=1000, learning_rate=0.05)

gbm.fit(train_X, train_y)
y_hat = gbm.predict(test_X)

# 下面是提交到kaggle上的格式

submission = pd.DataFrame({'row_id': test['row_id'], 'shop_id': y_hat})
print(submission)
submission.to_csv(r"./dataset/submission.csv", index=False)

     row_id    shop_id
0    118742   s_149642
1    118743     s_7301
2    118744   s_411850
3    118745   s_440043
4    118746   s_293082
5    118747   s_939447
6    118748   s_143910
7    118749     s_7301
8    118750   s_149642
9    118751  s_3743209
10   118752   s_293082
11   118753   s_411850
12   118754   s_411043
13   118755   s_411850
14   118756   s_411850
15   118757   s_677532
16   118758   s_149642
17   118759     s_7301
18   118760   s_143304
19   118761     s_7301
20   118762   s_143304
21   118763   s_143304
22   118764   s_411850
23   118765   s_411850
24   118766     s_7301
25   118767   s_143910
26   118768   s_288430
27   118769     s_7301
28   118770    s_12565
29   118771   s_288430
..      ...        ...
470  119212   s_143910
471  119213    s_10413
472  119214   s_149642
473  119215   s_149642
474  119216   s_440043
475  119217     s_7301
476  119218   s_149642
477  119219   s_410892
478  119220   s_410892
479  119221   s_143304
480  119222  s_3743209
481  119223

#### XGBoost参数说明
- [XGBoost参数介绍]( https://blog.csdn.net/m_buddy/article/details/79337492)